In [ ]:
import os
os.environ['CLMM_MODELING_BACKEND'] = 'ccl' # here you may choose ccl, nc (NumCosmo) or ct (cluster_toolkit)
import clmm
import numpy as np
import matplotlib.pyplot as plt

# Conversion from $200m$ to $500c$ overdensity definition

We define a cosmology first:

In [ ]:
cosmo = clmm.Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

## Define an NFW mass profile

### First SOD definition

We define halo parameters following the $200m$ overdensity definition:
1. the mass $M_{200m}$ 
2. the concentration $c_{200m}$

In [ ]:
#first SOD definition
model='nfw'
M1 = 1e14
c1 = 3
massdef1 = 'mean'
delta_mdef1 = 200
#cluster redshift
z_cl = 0.4

In [ ]:
#create a clmm Modeling object
nfw_def1 = clmm.Modeling(massdef=massdef1, delta_mdef=delta_mdef1, halo_profile_model=model)
#set the properties of the nfw profile
nfw_def1.set_mass(M1)
nfw_def1.set_concentration(c1)
nfw_def1.set_cosmo(cosmo)

Calculate the enclosed masses within r with the class method "eval_mass_in_radius". The calculaton can also be done in the functional interface with "compute_profile_mass_in_radius".

In [ ]:
r= np.logspace(-2, 0.4, 100)
#object oriented
nfw_def1_enclosed_oo = nfw_def1.eval_mass_in_radius(r3d=r, z_cl=z_cl)
#functional
nfw_def1_enclosed = clmm.compute_profile_mass_in_radius(r3d=r, redshift=z_cl, cosmo=cosmo,
                                                        mdelta=M1, cdelta=c1,
                                                        massdef=massdef1, delta_mdef=delta_mdef1,
                                                        halo_profile_model=model)

Compare the results given by the two functions:

In [ ]:
fig = plt.figure(figsize=(8, 6))
fig.gca().loglog(r, nfw_def1_enclosed, label='functional')
fig.gca().loglog(r, nfw_def1_enclosed_oo, ls='--', label='object oriented')
fig.gca().set_xlabel(r'$r\ [Mpc]$', fontsize = 20)
fig.gca().set_ylabel(r'$M_{NFW}(<r)\ [M_\odot]$', fontsize = 20)
fig.gca().legend()
plt.show()

We can compute the spherical overdensity radius, $r_{200m}$ with "eval_rdelta" in the object oriented interface or "compute_rdelta" in functional as the follwing:

In [ ]:
r200m_oo = nfw_def1.eval_rdelta(z_cl)
#functional interface
r200m = clmm.compute_rdelta(mdelta=M1, redshift=z_cl, cosmo=cosmo,
                            massdef=massdef1, delta_mdef=delta_mdef1)

print(f'r200m = {r200m_oo} Mpc')
print(f'r200m = {r200m} Mpc')

### Second SOD definition
We choose the second definition (here, we choose 500c)

In [ ]:
#2nd SOD definition
massdef2 = 'critical'
delta_mdef2 = 500

### Conversion from 200m to 500c

To find $M_2$ and $c_2$ of the second SOD definition, we solve the system of equations:
- $M_{M_1, c_1}(r_1) = M_{M_2, c_2}(r_1)$
- $M_{M_1, c_1}(r_2) = M_{M_2, c_2}(r_2)$

where $M_{M_i, c_i}(r)$ is the mass enclosed within a sphere of radius $r$ specified by the overdensity mass $M_i$ and concentration $c_i$. Here, $r_i$ is chosen to be the overdensity radius $r_{\Delta_i}$ of the $i$th overdensity definition, which is calculated with
$$
r_{\Delta_i} = \left(\frac{3M_{\Delta_i}}{4\pi \Delta_i \rho_{\rm bckgd,i}}\right)^{1/3}
$$
By identifying $M_{M_i, c_i}(r_i) = M_{\Delta_i}$ we now have two equations with two unknowns, $M_2$ and $c_2$:
- $M_1 = M_{M_2, c_2}(r_1)$
- $M_{M_1, c_1}(r_2) = M_2$

The conversion can be done by the "convert_mass_concentration" method of an NFW object and the output is the mass and concentration of the new NFW object in the second SOD definition.

In [ ]:
M2_oo, c2_oo = nfw_def1.convert_mass_concentration(z_cl=z_cl, massdef=massdef2, delta_mdef=delta_mdef2)
print(f'M2 = {M2_oo:.2e} M_sun')
print(f'c2 = {c2_oo:.2f}')

Similarly, there is functional interface to do the conversion

In [ ]:
M2, c2 = clmm.convert_profile_mass_concentration(
    mdelta=M1, cdelta=c1, redshift=z_cl, cosmo=cosmo,
    massdef=massdef1, delta_mdef=delta_mdef1, halo_profile_model=model,
    massdef2=massdef2, delta_mdef2=delta_mdef2)
print(f'M2 = {M2:.2e} M_sun')
print(f'c2 = {c2:.2f}')

To test the methods, we plot the differences between the 2 definitions of the same profile.
    $$
    \left|\frac{M_{M_2, c_2}(<r)}{M_{M_1, c_1}(<r)}-1\right|
    $$

In [ ]:
r = np.logspace(-3, 1, 300)
nfw_def2_enclosed = clmm.compute_profile_mass_in_radius(r3d=r, redshift=z_cl, cosmo=cosmo,
                                                        mdelta=M2, cdelta=c2,
                                                        massdef=massdef2, delta_mdef=delta_mdef2,
                                                        halo_profile_model=model)
fig2 = plt.figure(figsize=(8, 6))
fig2.gca().loglog(r, abs(nfw_def2_enclosed/nfw_def1.eval_mass_in_radius(r, z_cl)-1), ls='-')
fig2.gca().set_xlabel('r [Mpc]', fontsize = 20)
fig2.gca().set_ylabel(r'relative difference', fontsize = 20)
plt.show()